### Object Tracking

In [2]:
import cv2
import numpy as np

# camera is inirialised
capture = cv2.VideoCapture(0)

# range of HSV color space
lower_purple=np.array([130,50,90])
upper_purple = np.array([170,255,255])

# to store the center points
points=[]

# get frame size
ret,frame=capture.read()
Height,Width = frame.shape[:2]
frame_count=0

while True:
    # cam frame is read
    ret,frame = capture.read()
    hsv_img = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)

    # threshodl to get only purple colors
    mask =cv2.inRange(hsv_img,lower_purple,upper_purple)

    # find countors
    contours,_ = cv2.findContours(mask.copy(),cv2.RETR_EXTERNAL,
                                  cv2.CHAIN_APPROX_SIMPLE)
    # get the centroid of mass
    center = int(Height/2),int(Width/2)

    if len(contours)>0:
        # get only the lasrgest contour
        c=max(contours,key=cv2.contourArea)
        (x,y ),radius = cv2.minEnclosingCircle(c)
        M=cv2.moments(c)

        try:
            center=(int(M["m10"]/M["m00"]),int(M["m01"]/M["m00"]))
        except:
            center=int(Height/2),int(Width/2)

        # draw circle only if its greater than a particular size
        if radius>25:
            cv2.circle(frame,(int(x),int(y)),int(radius),(0,0,255))
            cv2.circle(frame,center,5,(0,255,0),-1)


    points.append(center)

    # loop over the points
    if radius>25:
        for i in range(1,len(points)):
            try:
                cv2.line(frame,points[i-1],points[i],(0,255,0),2)
            except:
                pass
        frame_count = 0

    else:
        # count the frames
        frame_count+=1
        # if its 10 frames then trails are deleted
        if frame_count==10:
            points=[]
            frame_count=0

    frame = cv2.flip(frame,1)
    cv2.imshow("Object tracker",frame)

    if cv2.waitKey(1)==13:
        break
capture.release()
cv2.destroyAllWindows()
